# Dumb Archival Legacy Investigations of Circumstellar Environments (ALICE), or DALICE

A quick and dirty JWST/NIRCam reference PSF library, inspired by ALICE (https://archive.stsci.edu/prepds/alice/) 

In [ ]:
from __future__ import division

# =============================================================================
# IMPORTS
# =============================================================================

import os
import pdb
import sys

import astropy.io.fits as pyfits
import matplotlib.pyplot as plt
import numpy as np

from spaceKLIP import database, coron1pipeline, coron2pipeline, coron3pipeline, pyklippipeline, imagetools, analysistools

# plotting
import matplotlib.pyplot as plt

## download data

In [ ]:
from spaceKLIP import mast

# this query will find all public reference stars taken in your given coronagraphic mask and filter. you can also make the query for MIRI, etc
table = mast.query_coron_datasets('NIRCam', 'F444W', 'MASKA335R', 'REF', ignore_exclusive_access=True, return_filenames=True, level='uncal')

print(f"Found {len(table)} unique dithers/star.")

In [ ]:
# we can check where the data comes from:
visid = np.unique(table['visit_id'])
pids = list(np.unique([int(x[2:6]) for x in visid]))
print(pids, sep=",")
print(list(np.unique(table['pi_name'])))

make sure you acknowledge where all the reference stars come from in your written work - im not sure if there's consensus on how to do this, but i plan to thank the PIs of these programs and cite whatever relevant papers might have been published from these datasets.

In [ ]:
# and then we can download the data to a directory of our choosing
dl_dir = '/Users/wbalmer/data/jwst/nircam335R444Wlibrary'
# mast.download_files(table, outputdir=dl_dir)

the rest of the notebook is just how I'd reduce the data using spaceKLIP, but fair warning, the package is under development and there will probably be other efforts to compile + implement RDI libraries in the future.

## load data

the code below assumes you've also independently downloaded your science (and maybe your science's associated references) into a folder we'll call "your_target_here"

In [ ]:
# Set the input and output directories and grab the input FITS files.

# some flags if we need to start/stop the reduction
reduced = False
cleanalign = False
aligned = False

# some spaceKLIP steps
blur = True
pad = False
coadd = True
crop = True

use_library = True

library_dir = '/Users/wbalmer/data/jwst/nircam335R444Wlibrary/'

science_dir = '/Users/wbalmer/data/jwst/your_target_here/'

reffiles = None

if aligned and coadd:
    input_dir = './spaceklip/coadded/'
    fitsfiles = sorted([input_dir + f for f in os.listdir(input_dir) if f.endswith('calints.fits')])
elif aligned:
    input_dir = './spaceklip/aligned/'
    fitsfiles = sorted([input_dir + f for f in os.listdir(input_dir) if f.endswith('calints.fits')])
elif cleanalign and crop:
    input_dir = './spaceklip/cropped/'
    fitsfiles = sorted([input_dir + f for f in os.listdir(input_dir) if f.endswith('calints.fits')])
elif cleanalign and blur:
    input_dir = './spaceklip/blurred/'
    fitsfiles = sorted([input_dir + f for f in os.listdir(input_dir) if f.endswith('calints.fits')])
elif reduced:
    input_dir = './spaceklip/stage2/'
    fitsfiles = sorted([input_dir + f for f in os.listdir(input_dir) if f.endswith('.fits')])
else:
    input_dir = science_dir
    fitsfiles = sorted([input_dir + f for f in os.listdir(input_dir) if f.endswith('uncal.fits')])
    if use_library:
        reffiles = sorted([library_dir + f for f in os.listdir(library_dir) if f.endswith('uncal.fits')])

if reffiles is not None:
    fitsfiles += reffiles

output_dir = './spaceklip/'

In [ ]:
# Initialize the spaceKLIP database and read the input FITS files.
Database = database.Database(output_dir=output_dir)
Database.read_jwst_s012_data(datapaths=fitsfiles,
                             psflibpaths=None,
                             bgpaths=None)

In [ ]:
# select a subset of obs, usually we are only interested in F200W+F444W

select_obs = [
              'JWST_NIRCAM_NRCALONG_F444W_MASKRND_MASK335R_SUB320A335R',
              ]

Database.obs = {k:Database.obs[k] for k in select_obs}

In [ ]:
if not reduced:
    coron1pipeline.run_obs(database=Database,
                           steps={'saturation': {'n_pix_grow_sat': 1,
                                                 'grow_diagonal': False},
                                  'refpix': {'odd_even_columns': True,
                                             'odd_even_rows': True,
                                             'nlower': 4,
                                             'nupper': 4,
                                             'nleft': 4,
                                             'nright': 4,
                                             'nrow_off': 0,
                                             'ncol_off': 0},
                                  'dark_current': {'skip': True},
                                  'persistence': {'skip': True},
                                  'jump': {'rejection_threshold': 4.,
                                           'three_group_rejection_threshold': 4.,
                                           'four_group_rejection_threshold': 4.,
                                           'maximum_cores': 'all'},
                                  'ramp_fit': {'save_calibrated_ramp': False,
                                               'maximum_cores': 'all'}},
                            subdir='stage1')

In [ ]:
if not reduced:
    coron2pipeline.run_obs(database=Database,
                           steps={'outlier_detection': {'skip': False}},
                           subdir='stage2')

In [ ]:
ImageTools = imagetools.ImageTools(Database)

In [ ]:
if not cleanalign:
    ImageTools.update_nircam_centers()

In [ ]:
if not cleanalign:
    ImageTools.subtract_median(types=['SCI', 'SCI_TA', 'SCI_BG', 'REF', 'REF_TA', 'REF_BG'],
                                   subdir='medsub')

In [ ]:
# Fix bad pixels using custom spaceKLIP routines. Multiple routines can be
# combined in a custom order by joining them with a + sign.
# - bpclean: use sigma clipping to find additional bad pixels.
# - custom: use custom map to find additional bad pixels.
# - timemed: replace pixels which are only bad in some frames with their
#            median value from the good frames.
# - dqmed:   replace bad pixels with the median of surrounding good
#            pixels.
# - medfilt: replace bad pixels with an image plane median filter.

if not cleanalign:
    ImageTools.fix_bad_pixels(method='bpclean+timemed+dqmed+medfilt',
                              bpclean_kwargs={'sigclip': 5,
                                              'shift_x': [-1, 0, 1],
                                              'shift_y': [-1, 0, 1]},
                              custom_kwargs={},
                              timemed_kwargs={},
                              dqmed_kwargs={'shift_x': [-1, 0, 1],
                                            'shift_y': [-1, 0, 1]},
                              medfilt_kwargs={'size': 4},
                              subdir='bpcleaned')

In [ ]:
if not cleanalign:
    ImageTools.replace_nans(cval=0.,
                            types=['SCI', 'SCI_BG', 'REF', 'REF_BG'],
                            subdir='nanreplaced')

In [ ]:
if not cleanalign:
    if blur:
        ImageTools.blur_frames()

In [ ]:
if not cleanalign:
    if crop:
        ImageTools.crop_frames(npix=10)
    
    if pad:
        ImageTools.pad_frames(
                              npix=[32, 33, 32, 33],
                              # npix=[1, 144, 73, 72], # shortwave
                              types=['SCI', 'SCI_BG', 'REF', 'REF_BG'],
                              cval=0.
                             )
        

In [ ]:
cc_method = 'fourier'
shift_method = 'spline'
algo = 'leastsq'

if not aligned:
    ImageTools.recenter_frames(
        # shift_method=shift_method,
        spectral_type='G2V',
        # shft_exp=1,
    )

In [ ]:
if not aligned:
    ImageTools.align_frames(
        method=cc_method,
        # shift_method=shift_method,
        align_algo=algo,
        # shft_exp=1,
        # mask_override=None,
        kwargs={},
        subdir='aligned'
    )

In [ ]:
if not aligned and coadd:
    ImageTools.coadd_frames()

## pyklip

In [ ]:
import platform
if platform.processor() == 'arm':
    os.environ["OPENBLAS_NUM_THREADS"] = "1"
    os.environ["OMP_NUM_THREADS"] = "1" 
# once other image reduction steps are done, run this so that pyklip.parallelized doesn't kill your apple chip

In [ ]:
pyklippipeline.run_obs(database=Database,
                       kwargs={'mode': ['ADI+RDI'],
                               'annuli': [1],
                               'subsections': [1],
                               'numbasis': [1, 2, 3, 4, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 120, 140, 160],
                               'algo': 'klip',
                               'save_rolls': False,
                              },
                       subdir='klipsub',
                      )

## contrast and companion forward modeling

In [ ]:
Analysis = analysistools.AnalysisTools(Database)

companions = [
              [-0.806, -0.758, 1e-6], # your candidate planets (or galaxies) here
             ]

companion_masks = [[-0.806, -0.758, 2], [0.8, -1.15, 4]]

starfile = './Your_star_here.vot'

mstar_err = 0.0


In [ ]:
inj_seps = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.5, 3.0]
Analysis.raw_contrast(starfile,spectral_type='G2V',companions=companions, plot_xlim=(0,3), subdir='rawcon')
Analysis.calibrate_contrast(
                            companions=companion_masks,
                            injection_seps=inj_seps,
                            plot_xlim=(0,3),
                            subdir='calcon'
                           )

In [ ]:
Analysis.extract_companions(companions, 
                            starfile, 
                            mstar_err,
                            spectral_type='G2V', 
                            fitmethod='mcmc',
                            fitkernel='matern32',
                            subdir='companions'
                           )